# Modelbuilder

This notebook illustrates the set up of a model to investigate NbS for coastal flooding. The example here presents a usecase for the Wash (United Kingdom) and combines input data from multiple sources. The model is set up using the HydroMT_sfincs python package. The usecase here requires a selection of its functionalities. If you want to learn more about building sfincs models this way, visit [HydroMT_sfincs - Modelbuilder example from script](https://github.com/Deltares/hydromt_sfincs/blob/main/examples/build_from_script.ipynb).

This modelbuilder combines many other Python packages. HydroMT_sfincs is used for the set up of the sfincs model, matplotlib for visualization, (rio)xarray to process netcdf files and geopandas to process geospatial data.



## Imports and user variables

In [ ]:
from shapely.geometry import Point
from os.path import join
import geopandas as gpd
import hydromt
from hydromt_sfincs import SfincsModel
import os
import shutil

In [ ]:
# Define the main directory and data library
# This is the directory where the data library is stored
data_libs = './input_dir/edito_sfincs_data.yml'
input_dir = './input_dir' 
main_dir = './wash_uk_example' 

# This is the directory where the model will be saved
sim_dir   = join(main_dir, 'sims')
if not os.path.exists(sim_dir):
    os.makedirs(sim_dir)

In [ ]:
# Define the input files
fn_domain       =  join(input_dir, 'domain.gpkg')
fn_domain_lines =  join(input_dir, 'domain_lines.gpkg')
fn_topo         = join(input_dir, 'delta_dtm_gebco_ref_msl.tif')
fn_veg          =  join(input_dir, 'da_veg.tif')
fn_storm_wl     =  join(input_dir, 'wl_ts.nc')

In [4]:
# Define the grid resolution
grid_resolution     = 50     # meters

# Define the roughness parameters
manning_saltmarshes = 0.08   # 0.07 Rezaie et al., 2020
manning_mangroves   = 0.15   # Zhang et al., 2012
manning_land        = 0.04   # default
manning_sea         = 0.02   # default

# Define waterlevel offset
slr_mean = 0.0

In [5]:
# Define the model observation points
obs_names             = ['veg1','veg2']
obs_pnts              = [Point(303082.833,5864791.153), 
                         Point(315739.571,5854677.962)]

## Initialization

In [ ]:
# Initialize hydromt DataCatalog
data_cat  = hydromt.DataCatalog(data_libs = data_libs)

# Initialize SFINCS model
mod = SfincsModel(root  = sim_dir+ '/sim_veg',  mode = 'w+', data_libs = data_libs)
gdf_domain       = gpd.read_file(fn_domain)
gdf_domain_lines = gpd.read_file(fn_domain_lines) 

Model dir already exists and files might be overwritten: p:\11209182-edito\06-Models\SFINCS_nbs_example\03_wash_uk_example\sims_test\sim_veg\gis.


## Grid & Topography

In [7]:
# setup grid from polygon in GeoDataFrame   
mod.setup_grid_from_region(region = {'geom':gdf_domain}, res = grid_resolution, rotated = True)
mod.config

# Define topography and vegetation datasets
da_topo   = data_cat.get_rasterdataset(fn_topo)
da_veg    = data_cat.get_rasterdataset(fn_veg)

# Assign topography to the model
datasets_dep  = [{"elevtn":da_topo}]
da_dep        = mod.setup_dep(datasets_dep=datasets_dep, buffer_cells= 1) # 

# set mask based on elevation (areas in km2)
mod.setup_mask_active(mask = gdf_domain , zmax = 10, drop_area = 10, fill_area = 100, reset_mask = True) 
mod.set_config("stopdepth", '250.0')
   
# Assign observation points to the model
gdf_obs               = gpd.GeoDataFrame({'names': obs_names}, geometry = obs_pnts, crs = mod.crs).set_index('names')
mod.setup_observation_points(gdf_obs)

## Boundary Conditions

In [8]:
# Define model boundaries
gdf_all_lines = gdf_domain_lines.loc[(gdf_domain_lines['bnd']==1)] 
if len(gdf_all_lines) > 0:
    gdf_all_lines = gdf_all_lines.loc[gdf_all_lines['geometry'] != None]
    gdf_all_lines.geometry = gdf_all_lines['geometry'].buffer(0.0075)
mod.setup_mask_bounds(btype = "waterlevel", include_mask = gdf_all_lines  , reset_bounds=True, zmax = 1) # here we mask the wl for all bnd points     

#update times, in line with wl forcing (times should be updated afeter setup_mask_bounds)
mod.set_config("tref"  , "20000101 000000")
mod.set_config("tstart", "20000101 000000")
mod.set_config("tstop" , "20000106 000000")
mod.set_config("dtout",  "600")


## Forcing

In [ ]:

mod.setup_waterlevel_forcing(geodataset = fn_storm_wl, offset=slr_mean , buffer = 15000)
df_ts = mod.forcing['bzs'].to_dataframe()
df_ts.index[~df_ts.isnull().any(axis=1)].tolist()[0]
df_ts.index[~df_ts.isnull().any(axis=1)].tolist()[-1]
t_start = str(df_ts.index[~df_ts.isnull().any(axis=1)].tolist()[0][0]).replace('-','').replace(':','')
t_end   = str(df_ts.index[~df_ts.isnull().any(axis=1)].tolist()[-1][0]).replace('-','').replace(':','')

#update times, in line with wl forcing
mod.set_config("tref"  , "20000101 000000")
mod.set_config("tstart", t_start)
mod.set_config("tstop" , t_end)
mod.set_config("dtout",  "600")
mod.set_config("dtmaxout","9999999")

## Add spatially varying roughness data

To incorporate the influence of different roughness for land, sea, and vegetation on flow this is added to the model based on our data.

In [11]:
# set increased Manning values for vegetation cover
mapping = {1:manning_saltmarshes, 2:manning_mangroves}
manning_veg_map = da_veg.copy()
for old_value, new_value in mapping.items():
    manning_veg_map = manning_veg_map.where(manning_veg_map != old_value, new_value)
  
datasets_rgh = [{'manning':manning_veg_map}]
mod.setup_manning_roughness(
           datasets_rgh=datasets_rgh,
           manning_land= manning_land,
           manning_sea= manning_sea,
           rgh_lev_land=0,  # the minimum elevation of the land
       )

## Write to file

In [ ]:
# Write the model with vegation to files
mod.write()

# create a copy for the model without vegetation for comparison
shutil.copytree(sim_dir + '/sim_veg', sim_dir + '/sim_noveg', dirs_exist_ok = True)
os.remove(sim_dir + '/sim_noveg/hydromt_data.yml')

# load the model and change manning file 
mod_noveg = SfincsModel(root  = sim_dir + '/sim_noveg',  mode = 'r+', data_libs = data_libs)
mod_noveg.config

mod_noveg.setup_manning_roughness(
              manning_land = manning_land,
              manning_sea  = manning_sea,
              rgh_lev_land = 0)  # the minimum elevation of the land

mod_noveg.write()

## Upload model to EDITO storage

In [ ]:
# upload the generated model to youe s3 bucket in "SFINCS_INPUT"
from upload_model import upload_model_to_s3_bucket
upload_model_to_s3_bucket(join(sim_dir, 'sim_noveg'))

## Run SFINCS simulation

In [ ]:
from IPython.display import display, Javascript, HTML
from urllib.parse import quote

# Encode URL
raw_url = (
    "https://datalab.dive.edito.eu/process-launcher/process-playground/"
    "sfincs-run-docker-test"
    "?name=sfincs-run-docker-test"
    "&version=0.2.3"
    "&s3=region-7e02ff37"
    "&resources.requests.cpu=«1800m»"
    "&resources.requests.memory=«5Gi»"
    "&resources.limits.cpu=«6400m»"
    "&resources.limits.memory=«28Gi»"
    "&autoLaunch=true"
)
safe_url = raw_url.replace("«", quote("«")).replace("»", quote("»"))

# Auto-open the new tab + show user prompt
display(HTML("<b>Launching process in a new tab... You can return here once it's finished.</b>"))

display(Javascript(f"""
    setTimeout(function() {{
        window.open("{safe_url}", "_blank");
    }}, 2000);
"""))

# Display the clickable fallback link
display(HTML(f'<a href="{safe_url}" target="_blank">🚀 Click here if auto-launch fails</a>'))

## Import simulation output

In [ ]:
import s3fs
# File
nc_file =   'sfincs_map.nc' #give the filename of the model output
directory = 'SFINCS_OUTPUT'  #define if input data is saved in subfolder of s3 bucket

##Import sfincs output
onxia_user_name = os.environ["GIT_USER_NAME"]
bucket_name = f"oidc-{onxia_user_name}"
S3_ENDPOINT_URL = os.environ["S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

# Download the nc-file
fs.download(f"{bucket_name}/{directory}/{nc_file}", join(sim_dir, 'sim_noveg', nc_file))
print(f"{bucket_name}/{directory}/{nc_file}")


## Visualize output